In [35]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

In [36]:
loader = PyPDFDirectoryLoader('Doc')

docs = loader.load()

In [37]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunk = splitter.split_documents(docs)

In [38]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [39]:
vector_store = FAISS.from_documents(chunk, embeddings)

In [40]:
retriever  = vector_store.as_retriever(search_type="mmr", search_kwargs={"k":2})

In [41]:
def merge_chunks(chunks):
    return "\n\n".join([chunk.page_content for chunk in chunks])

In [42]:
from langchain.prompts import PromptTemplate

template = """
You are a helpful and empathetic mental health assistant.

You will be given:
- A **context** (retrieved from mental health books/resources).
- A **user question**.

Guidelines:
1. If the answer IS present in the context, use ONLY the context to answer.
2. If the context does not contain the answer (or is empty), 
   say politely: "I couldn’t find this in my reference materials, 
   but based on my broader knowledge, here’s what I know:" 
   and then answer using your own understanding.
3. Keep the tone empathetic, supportive, and clear.
4. Never give harmful advice. If the user expresses crisis 
   (suicidal thoughts, self-harm), tell them to seek 
   immediate help from a professional or emergency services.

Context:
{context}

Question:
{question}

Answer:
"""

prompt = PromptTemplate(
    template=template, input_variables=["context", "question"]
)
